<a href="https://colab.research.google.com/github/Location-Artistry/GEO-DEV-NOTEBOOKS/blob/main/ArcREST_EXP_MAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ArcREST Data Explorer**   
# **<--  MAIN LIBRARY  -->**
# Efficient ArcGIS Server Browsing Library
## Quickly view all services and layers available from an ArcGIS Server URL
1-15 Added capabilities for NHBP ArcREST Server   
1-18 Added mapTable Class, fixed hanging function issue, most functions have try/except

In [ ]:
!pip install bmi-arcgis-restapi
import restapi

In [ ]:
# Class creates a <-mapServer object-> based on restapi.ArcServer from bmi-arcgis-restapi library
class mapServer:
# -init- function, ArcServer object as self.ags, server endpoint as self.url
  def __init__(self, url):
    try:
      creds = input(f'Credentials Needed (YES/NO)? ')
      if creds.upper() == 'YES':          
        id = input(f'ArcGIS Online USER ID: ')
        pwd = getpass.getpass('PASSWORD: ')
        self.ags = restapi.ArcServer(url, id, pwd)
      else:
        self.ags = restapi.ArcServer(url)
      self.url = url
      display('MAPSERVER OBJECT CREATED', self.ags)
    except:
      display(f'FAILED TO CREATE MAPSERVER')
# -getServices- displays and returns a list of all ArcGIS services
  def getServices(self):
    for i, service in enumerate(self.ags.services):
      display(f'{i} - Service Name: {service.name} -  Service Type: {service.type}')
    return self.ags.services 
# -getFolders- displays and returns list of all folders
  def getFolders(self):
    try:
      for i, folder in enumerate(self.ags.folders):
        display(f'{i} - Folder: {folder}') 
      return self.ags.folders
    except:
      return 0
# -getNestedServ- gets a mapServer's nested services
  def getNestedServices(self):
    for i, folder in enumerate(self.ags.folders):
      display(f'{i} - Folder: {folder}')
      ags = restapi.ArcServer(f'{self.ags.url}/{folder}')
      for z, service in enumerate(ags.services):
        display(f'   {z} - Service Name: {service.name} -  Service Type: {service.type}')
# -getAttr- get atrributes and show values for non-function methods of for MapServer Object
  def getAttr(self):
    for attribute in dir(self.ags):
      display(f'Attribute Names: {attribute} - {self.ags[attribute]}')
# -listAttr- list all attrbutes of MapServer Object
  def listAttr(self):
    return dir(self.ags)
# -getSubService- returns a arcService object within specified mapServer
  def getSubService(self, rootServer=0):
    folders = self.getFolders()
    services = self.getServices()
    display(f'{self.url} -  ')
    if not folders:
      fID = int(input(f'Select index of subservice: '))
      url = services[fID].name
      try:
        subService = self.getService(url)
        return subService
      except:
        url = (f'{self.url}/{url}/FeatureServer')
        # working here
        subService = arcService(self, url)
        return subService
    else:
      fID = int(input(f'Select index of subservice folder: '))
      serviceURL = (f'{self.url}/{folders[fID]}')
      subFolder = mapServer(serviceURL)
      subServices = subFolder.getServices()
      fID = int(input(f'Select index of subservice: '))
      serviceName = subServices[fID].name
      display(self.url, serviceName)
      subService = arcService(self, serviceName)
      return subService

# Class creates a <-arcService object-> based on ags.getService from bmi-arcgis-restapi library
class arcService:
# -init- create arcService from mapServer Class  
  def __init__(self, server, servName): 
    try:
      self.ags = server.ags.getService(servName)
      self.url = self.ags.url
      display(f'ARCSERVICE OBJECT CREATED - {self.ags.url}')  
    except:
      self.ags = restapi.MapService(servName)
      self.url = self.ags.url
      display(f'ARCSERVICE OBJECT CREATED - {self.ags.url}') 
  def getAttr(self):
    try:
      for attribute in dir(self.ags):
        display(f'Attribute Names: {attribute} - {self.ags[attribute]}')
    except:
      display(f'Could not get Attributes for: {self.ags.url}')
  def listAttr(self):
    try:
      return dir(self.ags)
    except:
      display(f'Could not get List Attributes for: {self.ags.url}')
  def desc(self):
    try:
      return self.ags.description
    except:
      display(f'Could not get Description for: {self.ags.url}')
  def spatialRef(self):
    try:
      return self.ags.spatialReference
    except:
      display(f'Could not get spatialReference for: {self.ags.url}')
  def units(self):
    try:
      return self.ags.units
    except:
      display(f'Could not get Units for: {self.ags.url}')
  def layers(self):
    try:
      return self.ags.list_layers()
    except:
      display(f'Could not list Layers for: {self.ags.url}')
  def tables(self):
    try:
      return self.ags.list_tables()
    except:
      display(f'Could not get Tables for: {self.ags.url}')
# -getLayer- returns a mapLayer from arcService Class
  def getLayer(self):
    try: 
      for i, lay in enumerate(self.ags.layers):
        display(f'{i} - {lay.name}')
      layers = self.layers()
      # small change here separate two lines of code for input ***
      time.sleep(1)
      fID = int(input('Select index of Layer: '))
      layerObj = mapLayer(self, layers[fID])
      return layerObj
    except:
      display(f'Could not Get Layers for: {self.ags.url}')
# -getTable- returns table object from arcService Class  
  def getTable(self):
    try:
      tableList = self.tables()
      for i, lay in enumerate(tableList):
        display(f'{i} - {tableList[i]}')
      time.sleep(1)
      fID = int(input(f'Select index of Table: '))      
      #table = self.ags.table(tableList[fID])
      tableID = tableList[fID]
      table = mapTable(self, tableID)
      return table
    except:
      display(f'Not able to return table for: {self.ags.url}')

# Class creates a <-Layer object-> based on ags.layer from bmi-arcgis-restapi library
class mapLayer:
  def __init__(self, service, lyrName): 
    try:
      self.lyr = service.ags.layer(lyrName)
      display(f'LAYER CREATED - {self.lyr.name} - id: {self.lyr.id} - url: {self.lyr.url}')
    except:
      display(f'COULD NOT CREATE mapLayer')
  def getInfo(self):
    try:
      display(f'{self.lyr.name} - capabilities: {self.lyr.capabilities} - query formats: {self.lyr.supportedQueryFormats} - count: {self.lyr.getCount()}')
    except:
      display(f'Could not get Info for: {self.lyr.name}')
  def getFields(self):
    try:
      return self.lyr.list_fields()
    except:
      display(f'Could not get Fields for: {self.lyr.name}')
  def getSubLayers(self):
    try:
      return self.lyr.subLayers
    except:
      display(f'{self.lyr.name} does not have any SubLayers')
# -layerQuery- prompts user for SQL query and fields, returns geoDataframe object which has access to all pandas/geoPandas methods
  def layerQuery (self):
    try:
      baseURL = self.lyr.url
      SQL = input(f'SQL query(ENTER for ALL): ') or '1=1'
      fields = input(f'Fields(ENTER for ALL)? ') or '*'
      RESTurl = (f'{baseURL}/query?f=geojson&where={SQL}&outFields={fields}')
      gdf = gpd.read_file(RESTurl)
      return gdf
    except:
      display(f'Query unsuccessful for: {self.lyr.name}')

# NEW CLASS NEED TO EDIT FIRST TO USE - MODELLED AFTER gisLayer
# Class creates a <-mapTable object-> based on ags.layer from bmi-arcgis-restapi library
class mapTable:
  def __init__(self, service, id): 
    try:
      self.table = service.ags.table(id)
      #self.lyr = service.ags.layer(lyrName)
      display(f'TABLE CREATED - {self.table.name} - id: {self.table.id} - url: {self.table.url}')
    except:
      display(f'COULD NOT CREATE mapTable')
  def getInfo(self):
    try:
      display(f'{self.table.name} - capabilities: {self.table.capabilities} - query formats: {self.table.supportedQueryFormats} - count: {self.table.getCount()}')
    except:
      display(f'Could not get Info for: {self.table.name}')
  def getFields(self):
    try:
      return self.table.list_fields()
    except:
      display(f'Could not get Fields for: {self.table.name}')
  def getSubLayers(self):
    try:
      return self.table.subLayers
    except:
      display(f'{self.table.name} does not have any SubTables')
# -layerQuery- prompts user for SQL query and fields, returns geoDataframe object which has access to all pandas/geoPandas methods
  def layerQuery (self):
    try:
      baseURL = self.table.url
      SQL = input(f'SQL query(ENTER for ALL): ') or '1=1'
      fields = input(f'Fields(ENTER for ALL)? ') or '*'
      RESTurl = (f'{baseURL}/query?f=geojson&where={SQL}&outFields={fields}')
      gdf = gpd.read_file(RESTurl)
      return gdf
    except:
      display(f'Query unsuccessful for: {self.table.name}')